In [95]:
import numpy as np
import random
import hashlib
from bisect import bisect, bisect_left

from typing import Callable

In [96]:
def alg1(K: int, L: int, R: Callable, n: int, h: Callable) -> [[int, int], ...]:
    # x = np.zeros((K, 2), dtype=int)
    x = [[0 for i in range(2)] for j in range(K)]
    max_allow_x = pow(2, n) - 1
    
    for i in range(K):
        x[i][0] = random.randint(0, max_allow_x)
        x_cur = x[i][0]
        for j in range(L):
            # x[i, j + 1] = h(R(x[i, j]))
            x_cur = h(R(x_cur))

        x[i][1] = x_cur

    return x

In [97]:
def bisect_left_castom(a, x, m, lo=0, hi=None, *, key=None,):
    if lo < 0:
        raise ValueError('lo must be non-negative')
    if hi is None:
        hi = len(a)
    # Note, the comparison uses "<" to match the
    # __lt__() logic in list.sort() and in heapq.
    if key is None:
        while lo < hi:
            mid = (lo + hi) // 2
            if a[mid] < x:
                lo = mid + 1
            else:
                hi = mid
    else:
        while lo < hi:
            mid = (lo + hi) // 2
            if key(a[mid]) % m < x % m:
                lo = mid + 1
            else:
                hi = mid
    return lo


def index(a, x, m):
    'Locate the leftmost value exactly equal to x'
    i = bisect_left_castom(a, x, m=m, key=lambda x: (x[1]))
    if i != len(a) and a[i][1] % m == x % m:
        return i
    
    return None

In [98]:
def alg2(K: int, L: int, R: Callable, x: [[int, int], ...], hash_val, n: int, h: Callable) -> int:
    m = pow(2, n)
    y = hash_val
    x_s = sorted(x, key=lambda x: x[1] % m)

    for j in range(L):
        id = index(x_s, y, m)
        if id is not None:
            x_ = x_s[id][0]

            for m_ in range(L - (j + 1)):
                x_ = h(R(x_))

            r_val = R(x_)
            if h(r_val) % m == hash_val % m:
                return r_val
            else:
                return None
        

        # for i in range(K):
        #     # if x[i][1] % m == y % m:
        #     if x[i][1] == y:
        #         x_ = x[i][0]

        #         for m_ in range(L - (j + 1)):
        #             x_ = h(R(x_))

        #         r_val = R(x_)
        #         if h(r_val) % m == hash_val % m:
        #             return r_val
        #         else:
        #             return None

        y = h(R(y))

    return None

In [99]:
def alg2_slow(K: int, L: int, R: Callable, x: [[int, int], ...], hash_val, n: int, h: Callable) -> int:
    m = pow(2, n)
    y = hash_val
    x_s = sorted(x, key=lambda x: x[1] % m)

    for j in range(L):
        for i in range(K):
            # if x[i][1] % m == y % m:
            if x[i][1] == y:
                x_ = x[i][0]

                for m_ in range(L - (j + 1)):
                    x_ = h(R(x_))

                r_val = R(x_)
                if h(r_val) % m == hash_val % m:
                    return r_val
                else:
                    return None

        y = h(R(y))

    return None

In [100]:
def sha_int_to_int(message: int) -> str:
    """return hash of int value message"""
    return int("0x" + hashlib.sha256(str(message).encode('ASCII')).hexdigest(), base=16)

# Task

In [101]:
def attack1(K: int, L: int, n: int, rand_v_hash):
    # _r = random.randint(0, pow(2, 128 - n) - 1)
    _r =  0x3762d7a3fb50e749963972a36f9c
    m = pow(2, n)

    def R(x: int):
        return _r + (x % m)
    
    x = alg1(K=K, L=L, R=R, n=n, h=sha_int_to_int)
    
    # for l in x:
    #     print(l)

    preimage = alg2(K=K, L=L, R=R, x=x, hash_val=rand_v_hash, n=n, h=sha_int_to_int)
    
    return preimage


In [102]:
if __name__ == "__main__":
    for _ in range(20):
        K = pow(2, 10)
        L = pow(2, 5)
        n = 16
        
        rand_v = random.randint(0, pow(2, 256) - 1)
        rand_v_hash = sha_int_to_int(rand_v)


        preimage = attack1(K=K, L=L, n=n, rand_v_hash=rand_v_hash)

        if preimage is None:
            print('"Прообраз не знайдено"')
        else:
            preimage_hash = sha_int_to_int(preimage)
            print(f"rand_x = {hex(rand_v)}")
            print(f"rand_x_hash = {hex(rand_v_hash)}")
            print(f"preimage = {hex(preimage)}")
            print(f"preimage_hash = {hex(preimage_hash)}")

"Прообраз не знайдено"
"Прообраз не знайдено"
rand_x = 0x68694bb0caec0f195dcb67cb2d81372f52cb9c3c4a5fc902cfc01b69f00f7561
rand_x_hash = 0xe02cb4859d1d3ce4828a2dcc6bcc37ec1c4d7ad4e67977c80d0eccfbafd3ac6c
preimage = 0x3762d7a3fb50e749963972a42b75
preimage_hash = 0x51465042601e75be69db05a7e5ff2e7a3dbc82652db137611c73b004ad4fac6c


"Прообраз не знайдено"
"Прообраз не знайдено"
"Прообраз не знайдено"
"Прообраз не знайдено"
"Прообраз не знайдено"
"Прообраз не знайдено"
"Прообраз не знайдено"
"Прообраз не знайдено"
rand_x = 0x3a916f1ef52f13e1b91223b8aeacaaf636a1eddf7fa5cdf55da4747c9d706b4d
rand_x_hash = 0xda5a4e047223eda1e42e657a9420d221628847fdf184bf2de86036f92c283727
preimage = 0x3762d7a3fb50e749963972a3c467
preimage_hash = 0x77af3f63f857ed4a72ad3e86a3191911ec25e69a4db375d7e9556fe74aa63727
"Прообраз не знайдено"
"Прообраз не знайдено"
"Прообраз не знайдено"
"Прообраз не знайдено"
"Прообраз не знайдено"
"Прообраз не знайдено"
"Прообраз не знайдено"
"Прообраз не знайдено"


In [103]:
def test_attack1(N: int):
    if N < 1:
        return [None]

    n = 16

    K_vals = [pow(2, 10), pow(2, 12), pow(2, 14)]
    L_vals = [pow(2, 5), pow(2, 6), pow(2, 7)]
    results = [[0 for j in range(3)] for i in range(3)]

    for i in range(3):
        for j in range(3):
            for k in range(N):
                rand_v = random.randint(0, pow(2, 256) - 1)
                rand_v_hash = sha_int_to_int(rand_v)
                preimage = attack1(K=K_vals[i], L=L_vals[j], n=n, rand_v_hash=rand_v_hash)

                if preimage is not None:
                    results[i][j] += 1
            
            results[i][j] = results[i][j] / N

    return results

In [104]:
rez = test_attack1(N = 0)

for l in rez:
    print(l)
        

None


# Task 2

In [105]:
def alg2_extended(K: int, L: int, tables: [[[int, int], ...], ...], r_values: [int, ...], hash_val, n: int, h: Callable) -> int:
    m = pow(2, n)
    y = [hash_val for i in range(K)]
    sorted_tables = []
    for table in tables:
        sorted_tables.append(sorted(table, key=lambda x: x[1] % m))

    # x_s = sorted(x, key=lambda x: x[1] % m)

    for j in range(L):
        for i in range(K):
            id = index(sorted_tables[i], y[i], m)
            if id is not None:
                x_ = sorted_tables[i][id][0]

                for m_ in range(L - (j + 1)):
                    x_ = h(r_values[i] + (x_ % m))

                r_val = r_values[i] + (x_ % m)
                if h(r_val) % m == hash_val % m:
                    return r_val
                else:
                    return None
            

            y[i] = h(r_values[i] + (y[i] % m))

    return None

In [106]:
def attack2(K: int, L: int, n: int, rand_v_hash):
    m = pow(2, n)
    
    tables = []
    r_values = []
    for i in range(K):
        _r = random.randint(0, pow(2, 128 - n) - 1)

        def R(x: int):
            return _r + (x % m)
        
        alg1_rez = alg1(K=K, L=L, R=R, n=n, h=sha_int_to_int)
        tables.append(alg1_rez)
        r_values.append(_r)


    # for l in x:
    #     print(l)

    preimage = alg2_extended(K=K, L=L, tables=tables, r_values=r_values, hash_val=rand_v_hash, n=n, h=sha_int_to_int)
    
    return preimage

In [107]:
if __name__ == "__main__":
    for _ in range(20):
        K = pow(2, 5)
        L = pow(2, 5)
        n = 16
        
        rand_v = random.randint(0, pow(2, 256) - 1)
        rand_v_hash = sha_int_to_int(rand_v)


        preimage = attack2(K=K, L=L, n=n, rand_v_hash=rand_v_hash)

        if preimage is None:
            print('"Прообраз не знайдено"')
        else:
            preimage_hash = sha_int_to_int(preimage)
            print(f"rand_x = {hex(rand_v)}")
            print(f"rand_x_hash = {hex(rand_v_hash)}")
            print(f"preimage = {hex(preimage)}")
            print(f"preimage_hash = {hex(preimage_hash)}")

"Прообраз не знайдено"
"Прообраз не знайдено"
rand_x = 0x379ffe9c10b456bf2c8ee9ddaf3b3c2fb8b4625ebec58219261638dbc8d35d83
rand_x_hash = 0x5586e36ad3aa6ffdd255fcd51bdf29da6c6f0dadd9b06cff532f43b45e609e2e
preimage = 0x7df938a7ae953600149ab29469d
preimage_hash = 0x1c9b32a1e6d4f23cea384297be89596ece14582985d8a10471d638b2cdb39e2e
"Прообраз не знайдено"
rand_x = 0x71706d441778158237b7105af9473f53f8d838509ea87d6802f3dfd016cc960c
rand_x_hash = 0xc12c0d51554f8c15438cc02e646f699e080d7577afaaf5f1c8ddb6faba13727b
preimage = 0x946336249927409b1dfd9c71387c
preimage_hash = 0x558cd215cf1cf2f6d67396939c0bebb6e7747d008985927119d4f4bbdacf727b
rand_x = 0xb7198404d0fba52bb88071edbe5369443a3ed2fa1690cee33c831bf1b6ab2b3d
rand_x_hash = 0x78cd9221f5809688e8fbbb35180f7e22c047ac39370398b2a88246ceaa97459b
preimage = 0x56ae0fe1a2e59ad0b51d11e2bac9
preimage_hash = 0x7264763386e0671c1c310a4dd61bceacfdefb6e2b4d4fbdccc22c13f7789459b
"Прообраз не знайдено"
"Прообраз не знайдено"
"Прообраз не знайдено"
"Прообраз не знай

In [108]:
def test_attack2(N: int):
    if N < 1:
        return [None]

    n = 16

    K_vals = [pow(2, 5), pow(2, 6), pow(2, 7)]
    L_vals = [pow(2, 5), pow(2, 6), pow(2, 7)]
    results = [[0 for j in range(3)] for i in range(3)]

    for i in range(3):
        for j in range(3):
            for k in range(N):
                rand_v = random.randint(0, pow(2, 256) - 1)
                rand_v_hash = sha_int_to_int(rand_v)
                preimage = attack2(K=K_vals[i], L=L_vals[j], n=n, rand_v_hash=rand_v_hash)

                if preimage is not None:
                    results[i][j] += 1
            
            results[i][j] = results[i][j] / N

    return results

In [109]:

from datetime import datetime

In [110]:
rez2 = test_attack2(N = 0)

for l in rez2:
    print(l)
        

None


# Halman Theorem (Probability)

In [160]:
from mpmath import mpf, mpc, mp, mpmathify, fadd, fdiv, fsub, fmul
from math import trunc

In [172]:
def calc_halman_probability(n, K, L):
    # m - K, t - L
    N = pow(2, n)

    sum = 0 
    for i in range(1, K + 1):
        for j in range(L):
            # sum += pow(mpmathify(1) - mpmathify(mpmathify(i * L) / mpmathify(N)), j + 1)
            p = fsub(1, fdiv(i * L, N))
            ppp = 1
            for i in range(j + 1):
                ppp = fmul(ppp, p)

            sum = fadd(sum, ppp)

    return fdiv(sum, N)

In [173]:
# def calc_halman_probability(n, K, L):
#     # m - K, t - L
#     N = pow(2, n)

#     sum = 0 
#     for i in range(1, K + 1):
#         p = fsub(1, fdiv(i * L, N))

#         ppp = 1
#         for t in range(L + 1):
#             ppp = fmul(ppp, p)

#         sum = fadd(sum, fdiv(fsub(1, ppp), fsub(1, p)))
#         sum = fsub(sum, 1)


#     return fdiv(sum, N)

In [176]:
K_vals = [pow(2, 10), pow(2, 12), pow(2, 14)]
L_vals = [pow(2, 5), pow(2, 6), pow(2, 7)]
results = [[0 for j in range(3)] for i in range(3)]

for i in range(3):
    for j in range(3):
        p = calc_halman_probability(n=16, K=K_vals[i], L=L_vals[j])
        # print(p)
        print(f"{int(p* 100)}\% & ", end="")
    
    print("")

0\% & 0\% & 0\% & 
0\% & 0\% & 0\% & 
0\% & 0\% & 0\% & 


In [177]:
K_vals = [pow(2, 5), pow(2, 6), pow(2, 7)]
L_vals = [pow(2, 5), pow(2, 6), pow(2, 7)]
results = [[0 for j in range(3)] for i in range(3)]

for i in range(3):
    for j in range(3):
        p = calc_halman_probability(n=16, K=K_vals[i], L=L_vals[j])
        results[i][j] = 1 - pow(1 - p, K_vals[i])



print("K tables probabilities")
for l in results:
    for i in l:
        print(f"{int(i * 100)}\% & ", end="")
    
    print("")

K tables probabilities
35\% & 36\% & 27\% & 
82\% & 83\% & 72\% & 
99\% & 99\% & 99\% & 
